In [2]:
!pip install sahi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 94 kB 1.6 MB/s 
     |████████████████████████████████| 87 kB 6.3 MB/s 
     |████████████████████████████████| 97 kB 3.8 MB/s 
     |████████████████████████████████| 3.2 MB 52.5 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=fff52e5aa29aee98b89de7994f0d55b6906a3b8f0433d8a1130910bd1f5151c9
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that a

In [3]:
from sahi.utils.coco import Coco, CocoCategory, CocoImage, CocoAnnotation
from sahi.utils.file import save_json
from tqdm import tqdm
import os 

In [4]:
IMAGE_W = 512
IMAGE_H = 512
def yoloToCOCO(width_yolo, height_yolo, x_yolo, y_yolo):
    width_coco = width_yolo * IMAGE_W
    height_coco = height_yolo * IMAGE_H
    x_coco = x_yolo * IMAGE_W - (width_coco/2)
    y_coco = y_yolo * IMAGE_H - (height_coco/2)
    return x_coco,y_coco,width_coco,height_coco

def createJSON(coco, labels, save_path):
    for l in tqdm(labels):
        name = l[:-4]
        if not os.path.exists(os.path.join(dataPath, 'images', name+'.jpg')):
            continue
        coco_image = CocoImage(file_name=name+".jpg", height=IMAGE_H, width=IMAGE_W)
        lbl_file = os.path.join(dataPath, 'labels',l)
        with open(lbl_file) as f:
            lines = f.readlines()
        lines = [[float(j) for j in i.strip().split()] for i in lines]
        for c,x,y,w,h in lines:
            x_min,y_min,width,height = yoloToCOCO(w,h,x,y)
            if c==0:
                coco_image.add_annotation(
                    CocoAnnotation(
                    bbox=[x_min, y_min, width, height],
                    category_id=0,
                    category_name='elkhorn'
                    )
                )
            else:
                coco_image.add_annotation(
                    CocoAnnotation(
                    bbox=[x_min, y_min, width, height],
                    category_id=1,
                    category_name='staghorn'
                    )
                )
        coco.add_image(coco_image)
    save_json(data=coco.json, save_path=save_path)
    return coco

In [6]:
coco = Coco()
coco.add_category(CocoCategory(id=0, name='elkhorn'))
coco.add_category(CocoCategory(id=1, name='staghorn'))

dataPath = '/content/drive/MyDrive/yolo-dataset/train'
images = os.listdir(os.path.join(dataPath, 'images'))
labels = os.listdir(os.path.join(dataPath, 'labels'))

print(f'Num images: {len(images)}, num labels: {len(labels)}')
save_path = "/content/drive/MyDrive/yolo-dataset/train_coco.json"
coco = createJSON(coco = coco, labels = labels, save_path = save_path)

FileNotFoundError: ignored

In [ ]:
coco = Coco()
coco.add_category(CocoCategory(id=0, name='elkhorn'))
coco.add_category(CocoCategory(id=1, name='staghorn'))

dataPath = '/content/drive/MyDrive/yolo-dataset/valid'
images = os.listdir(os.path.join(dataPath, 'images'))
labels = os.listdir(os.path.join(dataPath, 'labels'))

print(f'Num images: {len(images)}, num labels: {len(labels)}')
save_path = "/content/drive/MyDrive/yolo-dataset/valid_coco.json"
coco = createJSON(coco = coco, labels = labels, save_path = save_path)

Num images: 291, num labels: 291


100%|██████████| 291/291 [00:01<00:00, 288.37it/s]


In [ ]:
coco = Coco()
coco.add_category(CocoCategory(id=0, name='elkhorn'))
coco.add_category(CocoCategory(id=1, name='staghorn'))

dataPath = '/content/drive/MyDrive/yolo-dataset/test'
images = os.listdir(os.path.join(dataPath, 'images'))
labels = os.listdir(os.path.join(dataPath, 'labels'))

print(f'Num images: {len(images)}, num labels: {len(labels)}')
save_path = "/content/drive/MyDrive/yolo-dataset/test_coco.json"
coco = createJSON(coco = coco, labels = labels, save_path = save_path)

Num images: 222, num labels: 222


100%|██████████| 222/222 [00:00<00:00, 289.62it/s]


In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import requests
from pycocotools.coco import COCO

def main():

    coco_annotation_file_path = save_path

    coco_annotation = COCO(annotation_file=coco_annotation_file_path)

    # Category IDs.
    cat_ids = coco_annotation.getCatIds()
    print(f"Number of Unique Categories: {len(cat_ids)}")
    print("Category IDs:")
    print(cat_ids)  # The IDs are not necessarily consecutive.

    # All categories.
    cats = coco_annotation.loadCats(cat_ids)
    cat_names = [cat["name"] for cat in cats]
    print("Categories Names:")
    print(cat_names)

    # Category ID -> Category Name.
    query_id = cat_ids[0]
    query_annotation = coco_annotation.loadCats([query_id])[0]
    query_name = query_annotation["name"]
    query_supercategory = query_annotation["supercategory"]
    print("Category ID -> Category Name:")
    print(
        f"Category ID: {query_id}, Category Name: {query_name}, Supercategory: {query_supercategory}"
    )

    # Category Name -> Category ID.
    query_name = cat_names[1]
    query_id = coco_annotation.getCatIds(catNms=[query_name])[0]
    print("Category Name -> ID:")
    print(f"Category Name: {query_name}, Category ID: {query_id}")

    # Get the ID of all the images containing the object of the category.
    img_ids = coco_annotation.getImgIds(catIds=[query_id])
    print(f"Number of Images Containing {query_name}: {len(img_ids)}")

    # Pick one image.
    img_id = img_ids[2]
    img_info = coco_annotation.loadImgs([img_id])[0]
    img_file_name = img_info["file_name"]
    img_file = os.path.join(dataPath,'images',img_file_name)
    # img_url = img_info["coco_url"]
    print(
        f"Image ID: {img_id}, File Name: {img_file_name}"
    )

    # Get all the annotations for the specified image.
    ann_ids = coco_annotation.getAnnIds(imgIds=[img_id], iscrowd=None)
    anns = coco_annotation.loadAnns(ann_ids)
    print(f"Annotations for Image ID {img_id}:")
    print(anns)

    # Use URL to load image.
    im = Image.open(img_file)

    # Save image and its labeled version.
    plt.axis("off")
    plt.imshow(np.asarray(im))
    plt.savefig(f"{img_id}.jpg", bbox_inches="tight", pad_inches=0)
    # Plot segmentation and bounding box.
    coco_annotation.showAnns(anns, draw_bbox=True)
    plt.savefig(os.path.join(dataPath,f"{img_id}_annotated.jpg"), bbox_inches="tight", pad_inches=0)

    return

In [1]:
main()

NameError: ignored